In [55]:
# !pip install pyvi

In [2]:
import pandas as pd
from tqdm import tqdm
import numpy as np
from pyvi import ViTokenizer, ViPosTagger
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras import layers, models, callbacks, preprocessing, losses, optimizers, utils

In [57]:
# from google.colab import drive
# drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [58]:
# %cd /content/drive/My Drive/Colab Notebooks/Text_Generator

/content/drive/My Drive/Colab Notebooks/Text_Generator


In [59]:
# %ls

data/                  scrape_news.ipynb           temp/                      word_embedding.ipynb
data_processing.ipynb  search_generate_text.ipynb  text-classification.ipynb
model/                 src/                        text_generator.ipynb


In [3]:
vnexpress = pd.read_csv('./data/vnexpress/csv/vnexpress.csv').drop(columns=['Unnamed: 0'], axis=1)
vnexpress.shape

(1347, 6)

In [5]:
vnexpress

,article_id,content,topic,sub-topic,title,description
0,0,Người mẹ 42 tuổi ở Đà Nẵng vội vã tra hỏi và v...,doi-song,to-am,Cha mẹ học về giáo dục giới tính để dạy con,Nghe con trai 14 tuổi nói đã làm cho chị hàng ...
1,1,"Vợ chồng người phụ nữ ở Mỹ Đức, Hà Nội luôn tự...",doi-song,to-am,Nỗi lòng của phụ huynh có con mang thai tuổi đ...,Khi con gái đang là sinh viên năm nhất một trư...
2,2,"Kati Morton, nhà trị liệu hôn nhân và gia đình...",doi-song,to-am,Giải mã 'Hội chứng con gái đầu lòng',"""Hội chứng con gái đầu lòng"" cho rằng con gái ..."
3,3,"Anh Thắng, chủ tịch Hội làm vườn tỉnh Nghệ An ...",doi-song,to-am,Vườn sân thượng như khu rừng giải nhiệt mùa hè,Nghệ AnMuốn hạ nhiệt cho ngôi nhà khi nắng nón...
4,4,"Vừa kết thúc môn tiếng Anh, Phương bị sốt cao,...",doi-song,to-am,Nghị lực của cô bé ung thư,"Hà NamMột tuần trước, cô bé 14 tuổi từ Bệnh vi..."
...,...,...,...,...,...,...
1342,1342,"Từ ngày 4/4, xe khách trên 30 chỗ, xe giường n...",thoi-su,giao-thong,"Vì sao cấm xe tải nặng, xe khách đi cao tốc Ca...","Cục Đường bộ Việt Nam cấm xe tải nặng, xe khác..."
1343,1343,Dự án đang được UBND tỉnh xem xét trình HĐND t...,thoi-su,giao-thong,Gần 1.000 tỷ nâng cấp đường nối Phan Thiết với...,Bình Thuận11 km đường tỉnh 719 từ Trần Quý Cáp...
1344,1344,"Ngày 1/4, Ban An toàn giao thông tỉnh Thừa Thi...",thoi-su,giao-thong,Hai tỉnh không đồng tình cấm xe trên 30 tấn đi...,Thừa Thiên Huế và Quảng Trị lo ngại cấm xe tải...
1345,1345,Nhằm đồng bộ các dự án cao tốc Bắc Nam 4 làn x...,thoi-su,giao-thong,Hơn 10.000 tỷ đồng đầu tư mở rộng cao tốc hai ...,Hai dự án cao tốc Cam Lộ - La Sơn và La Sơn - ...


In [6]:
vnexpress['tag'] = vnexpress['content'] + vnexpress['title'] + vnexpress['description']
vnexpress.drop(columns=['content', 'title', 'description'], inplace=True)

In [7]:
mask = vnexpress['tag'].isna()
vnexpress[mask]

,article_id,topic,sub-topic,tag
93,93,doi-song,bai-hoc-song,NaN
119,119,doi-song,bai-hoc-song,NaN
214,214,doi-song,nha,NaN
324,324,doi-song,tieu-dung,NaN
357,357,doi-song,tieu-dung,NaN
504,504,giai-tri,phim,NaN
1163,1163,thoi-su,lao-dong-viec-lam,NaN
1179,1179,thoi-su,dan-sinh,NaN
1183,1183,thoi-su,dan-sinh,NaN
1192,1192,thoi-su,dan-sinh,NaN


In [8]:
vnexpress.dropna(axis=0, inplace=True)
vnexpress.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1337 entries, 0 to 1346
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   article_id  1337 non-null   int64 
 1   topic       1337 non-null   object
 2   sub-topic   1337 non-null   object
 3   tag         1337 non-null   object
dtypes: int64(1), object(3)
memory usage: 52.2+ KB


In [9]:
news = vnexpress.copy()
label_encoder = LabelEncoder()
news['topic'] = label_encoder.fit_transform(vnexpress['topic'])

print(news['topic'].value_counts())
news

topic
1    616
0    360
3    350
2     11
Name: count, dtype: int64


,article_id,topic,sub-topic,tag
0,0,0,to-am,Người mẹ 42 tuổi ở Đà Nẵng vội vã tra hỏi và v...
1,1,0,to-am,"Vợ chồng người phụ nữ ở Mỹ Đức, Hà Nội luôn tự..."
2,2,0,to-am,"Kati Morton, nhà trị liệu hôn nhân và gia đình..."
3,3,0,to-am,"Anh Thắng, chủ tịch Hội làm vườn tỉnh Nghệ An ..."
4,4,0,to-am,"Vừa kết thúc môn tiếng Anh, Phương bị sốt cao,..."
...,...,...,...,...
1342,1342,3,giao-thong,"Từ ngày 4/4, xe khách trên 30 chỗ, xe giường n..."
1343,1343,3,giao-thong,Dự án đang được UBND tỉnh xem xét trình HĐND t...
1344,1344,3,giao-thong,"Ngày 1/4, Ban An toàn giao thông tỉnh Thừa Thi..."
1345,1345,3,giao-thong,Nhằm đồng bộ các dự án cao tốc Bắc Nam 4 làn x...


In [10]:
news = news.sample(frac=1).reset_index().drop(columns=['index'])
news

,article_id,topic,sub-topic,tag
0,16,0,to-am,"Cũng bởi tình yêu này, cô gái 23 tuổi muốn bản..."
1,459,1,phim,Megalopolis Tác phẩm của đạo diễn Godfather Fr...
2,943,1,san-khau-my-thuat,Tranh gây chú ý khi được Antoine Kruk đăng lên...
3,1149,3,lao-dong-viec-lam,"Thông tin được ông Lê Văn Thinh, Giám đốc Sở L..."
4,104,0,bai-hoc-song,Dạy trẻ làm việc nhà từ nhỏ Julie Lythcott-Hai...
...,...,...,...,...
1332,707,1,thoi-trang,"Theo Lifestyle Asia, hãng Julien's Auction giớ..."
1333,839,1,sach,"Tối 18/4, ông Dương Thanh Hoài - đại diện ban ..."
1334,1226,3,dan-sinh,"Từ 6h ngày 14/2, sau khi tế lễ, các cao niên v..."
1335,1237,3,dan-sinh,"Ngày cận Tết Giáp Thìn, cơ sở của bà Nguyễn Th..."


In [11]:
sequences = news['tag'].tolist()
sequences[269]

'Câu chuyện của Tôn Ca, người đàn ông 40 tuổi đến từ tỉnh Liêu Ninh, đã gây chú ý trên nhiều trang mạng xã hội Trung Quốc giữa tháng 3/2024. Theo đó hai năm trước, cha của Tôn được chẩn đoán mắc ung thư biểu mô tuyến ruột thừa, một căn bệnh hiếm gặp và tỷ lệ mắc rất thấp. Để cứu sống cha, người con trai đã đưa ông đi khắp các bệnh viện trong nước, thậm chí ra nước ngoài điều trị nhưng không có kết quả. "Cuộc sống của ông cứ dần tàn lụi trước mắt tôi", Tôn kể. Người cha đã mất vào tháng 5/2023 sau nhiều đợt hóa trị. Hiểu rằng bà nội 91 tuổi - người đang điều trị bệnh tim trong bệnh viện - không thể trụ nổi nếu biết tin con trai mất nên Tôn bàn với gia đình giấu kín thông tin. Mỗi lần về thăm bà, cháu trai đều thông báo, bố anh đang điều trị trong một viện lớn ở Bắc Kinh, sức khỏe dần ổn định. Nhưng đến Tết 2024, vì quá nhớ con trai nên người bà đòi đi Bắc Kinh. Lúc này Tôn lấy lý do, vì phải điều trị căn bệnh đặc biệt nên bác sĩ hạn chế người đến thăm nom. Thấy bà dần nghi ngờ, anh hứa 

In [12]:
# filter several punctuations in 50803 sentences
tokenizer = preprocessing.text.Tokenizer(filters='!“"”#$%&()*+,-./:;<=>?@[\]^`{|}~ ')
# create vocabulary (corpus) for each word in 50803 sentences
tokenizer.fit_on_texts(sequences)

print(len(tokenizer.word_index))
tokenizer.word_index

14697


{'và': 1,
 'của': 2,
 'có': 3,
 'trong': 4,
 'người': 5,
 'được': 6,
 'là': 7,
 'cho': 8,
 'với': 9,
 'không': 10,
 'các': 11,
 'một': 12,
 'năm': 13,
 'công': 14,
 'để': 15,
 'khi': 16,
 'nhiều': 17,
 'ở': 18,
 'những': 19,
 'nhà': 20,
 'từ': 21,
 'làm': 22,
 'đến': 23,
 'thể': 24,
 'theo': 25,
 'về': 26,
 'đồng': 27,
 'thành': 28,
 'động': 29,
 'đầu': 30,
 'ông': 31,
 'việc': 32,
 'anh': 33,
 'con': 34,
 'hơn': 35,
 'gia': 36,
 'như': 37,
 'ra': 38,
 'trên': 39,
 'cô': 40,
 'tại': 41,
 'ngày': 42,
 'vào': 43,
 'sự': 44,
 'hiện': 45,
 'tôi': 46,
 'nhân': 47,
 'sau': 48,
 'đó': 49,
 'thời': 50,
 'nói': 51,
 'cũng': 52,
 'này': 53,
 'đã': 54,
 'hai': 55,
 'nước': 56,
 'tác': 57,
 'lại': 58,
 'cao': 59,
 'sĩ': 60,
 'tuổi': 61,
 'diễn': 62,
 'nam': 63,
 'biết': 64,
 'cách': 65,
 'viên': 66,
 'qua': 67,
 'hàng': 68,
 'ảnh': 69,
 'sẽ': 70,
 'đi': 71,
 'học': 72,
 'bộ': 73,
 'còn': 74,
 'gian': 75,
 'thông': 76,
 'việt': 77,
 'nên': 78,
 'hội': 79,
 'nhất': 80,
 'số': 81,
 'quốc': 82,
 'phẩm

In [13]:
max_length = news['tag'].map(len).max()
max_length

16304

In [14]:
sequence_digit = tokenizer.texts_to_sequences(sequences)
X = np.array(preprocessing.sequence.pad_sequences(sequence_digit, maxlen=max_length, padding='pre'))

print(len(X[0]))
print(X[0:2])

16304
[[   0    0    0 ...  497 2902  412]
 [   0    0    0 ...   18 2048  640]]


In [15]:
topic_size = len(news['topic'].unique()) + 1
topic_size

5

In [17]:
## Sửa định dạng y
y = np.array([news['topic']])
y = y.reshape(-1, 1)
y = utils.to_categorical(y, num_classes=topic_size)

print(y[0:5])

[[1. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0.]
 [0. 1. 0. 0. 0.]
 [0. 0. 0. 1. 0.]
 [1. 0. 0. 0. 0.]]


In [18]:
vocab_size = len(tokenizer.word_index) + 1
vocab_size

14698

In [23]:
model = models.Sequential([
  layers.Embedding(vocab_size, 50, input_shape=(max_length,)),
  layers.LSTM(64),
  layers.Dense(64, activation='relu'),
  layers.Dense(5, activation='softmax')
])

model.summary()

d:\Programming\Vietnamese-Text-Generator\.venv\Lib\site-packages\keras\src\layers\core\embedding.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model: "sequential_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_3 (Embedding)         │ (None, 16304, 50)      │       734,900 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_3 (LSTM)                   │ (None, 64)             │        29,440 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_6 (Dense)                 │ (None, 64)             │         4,160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_7 (Dense)                 │ (None, 5)              │           325 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 768,825 (2.93 MB)

 Trainable params: 768,825 (2.93 MB)

 Non-trainable params: 0 (0.00 B)

In [24]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [25]:
model.fit(X, y, epochs=25, batch_size=64)

Epoch 1/25
21/21 ━━━━━━━━━━━━━━━━━━━━ 580s 28s/step - accuracy: 0.3987 - loss: 1.5379
Epoch 2/25
 6/21 ━━━━━━━━━━━━━━━━━━━━ 7:41 31s/step - accuracy: 0.5009 - loss: 1.0668

In [124]:
sample_text = ['"Tôi là niềm hy vọng của gia đình khi có thêm con trai nối dõi cũng như bố mẹ có người chăm sóc khi về già", Lưu Vũ Hằng, 30 tuổi, kỹ sư IT ở Vũ Hán, tỉnh Hồ Bắc nói. Lưu nhận ra mình không nhận được nhiều sự quan tâm và yêu thương từ cha mẹ. Nhưng suốt 30 năm cuộc đời, dù chọn lựa gì người đàn ông này cũng phải đặt câu hỏi "Nếu chọn công việc này có thời gian chăm cha mẹ không"; "Nếu sang tỉnh khác sinh sống, cha mẹ ai sẽ chăm lo". Bố mẹ già, sự khác biệt thế hệ quá lớn nên họ không thể cho con trai út lời khuyên, hướng dẫn phù hợp tại những thời điểm quan trọng của cuộc đời. Lưu bắt đầu cảm thấy lạc lõng với cha mẹ khi bị một bạn cùng lớp bạo hành thời tiểu học. Cậu bé 9 tuổi khi đó không dám kể lại bởi biết bố mẹ sẽ phán xét ngược rằng "không có lửa làm sao có khói, chắc con phải làm gì thì đứa kia mới thế". Trong 10 năm tiếp theo, Lưu lần lượt từ bỏ cơ hội học cao học cũng như đến thành phố lớn làm việc bởi lý do bố mẹ quá già, cần người bên cạnh. Nhưng điều khiến người đàn ông 30']
sequences_digit = tokenizer.texts_to_sequences(sample_text)
X_test = np.array(preprocessing.sequence.pad_sequences(sequences_digit, maxlen=max_length, padding='pre'))
X_test

predictions = model.predict(X_test)
print(predictions)

1/1 [==============================] - 1s 622ms/step
[[3.3310443e-04 9.9966502e-01 1.9105621e-06]]


In [128]:
result = label_encoder.inverse_transform([np.argmax(predictions[0])])
result

array(['thoi-su'], dtype=object)